In [7]:
# Step 1: Load CSV and inspect
import pandas as pd

# Load CSV
df = pd.read_csv("mock_ads_deep_training_data.csv")  # replace with your actual CSV file path

# Show first few rows
print(df.head())

# Info about columns and types
print(df.info())

# Check for missing values
print(df.isnull().sum())

# Basic statistics
print(df.describe())


       platform  campaign_id campaign_name  impressions  clicks    spend  \
0  LinkedIn Ads       300001  Campaign_A_0        29230     326  2454.81   
1  Facebook Ads       100002  Campaign_B_1        41677    6729  2930.53   
2  Facebook Ads       100003  Campaign_C_2        11087    1358   327.95   
3  Facebook Ads       100004  Campaign_D_3        23231    1183  2864.28   
4  Facebook Ads       100005  Campaign_E_4        15192     771  1625.96   

   conversions    CTR   ROAS  campaign_score  performance_alerts  \
0           61   1.12   0.59              21                   1   
1         1311  16.15  14.58             176                   0   
2          581  12.25  48.30             155                   0   
3          245   5.09   1.77              63                   0   
4            4   5.08   0.07              41                   0   

                          budget_recommendation  \
0  Decrease budget or pause underperforming ads   
1                       Maintain

In [8]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
import re  # for extracting percentages safely

# Copy your dataframe
df_model = df.copy()

# -------------------------------
# 1. Encode platform (categorical input)
# -------------------------------
platform_encoder = LabelEncoder()
df_model['platform_encoded'] = platform_encoder.fit_transform(df_model['platform'])

# -------------------------------
# 2. Encode categorical outputs
# -------------------------------
# Budget Recommendation
budget_rec_encoder = LabelEncoder()
df_model['budget_rec_encoded'] = budget_rec_encoder.fit_transform(df_model['budget_recommendation'])

# Audience Expansion Opportunity (Yes/No)
audience_exp_encoder = LabelEncoder()
df_model['audience_exp_encoded'] = audience_exp_encoder.fit_transform(df_model['audience_expansion_opportunity'])

# Bid Strategy Optimization
bid_strategy_encoder = LabelEncoder()
df_model['bid_strategy_encoded'] = bid_strategy_encoder.fit_transform(df_model['bid_strategy_optimization'])

# -------------------------------
# 3. Process recommended_budget_distribution
# Example: "22% Alpha, 38% Beta, 40% Gamma" -> [22,38,40]
# -------------------------------
def parse_budget_distribution(text):
    parts = text.split(',')
    percentages = []
    for part in parts:
        percent = float(part.strip().split('%')[0])
        percentages.append(percent)
    return percentages

budget_dist = df_model['recommended_budget_distribution'].apply(parse_budget_distribution)
budget_dist_df = pd.DataFrame(budget_dist.tolist(), columns=['budget_alpha', 'budget_beta', 'budget_gamma'])
df_model = pd.concat([df_model, budget_dist_df], axis=1)

# -------------------------------
# 4. Features (inputs) for model
# -------------------------------
X = df_model[['platform_encoded', 'impressions', 'clicks', 'spend', 'conversions', 'CTR', 'ROAS',
              'budget_alpha', 'budget_beta', 'budget_gamma']].values

# -------------------------------
# 5. Targets (outputs)
# -------------------------------

# Safe extraction of numeric percentage from budget_reallocation_opportunity
def extract_percentage(text):
    if isinstance(text, str):
        match = re.search(r'(\d+)%', text)
        if match:
            return float(match.group(1))
    return 0.0

y_campaign_score = df_model['campaign_score'].values
y_performance_alerts = df_model['performance_alerts'].values
y_budget_rec = df_model['budget_rec_encoded'].values
y_roi_forecast = df_model['ROAS'].values  # can be replaced with advanced ROI formula
y_budget_dist_total = df_model[['budget_alpha','budget_beta','budget_gamma']].values
y_avg_perf_score = df_model['avg_performance_score'].values
y_budget_realloc = df_model['budget_reallocation_opportunity'].apply(extract_percentage).values
y_audience_exp = df_model['audience_exp_encoded'].values
y_bid_strategy = df_model['bid_strategy_encoded'].values

# -------------------------------
# 6. Standardize features
# -------------------------------
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print("Feature shape:", X_scaled.shape)
print("Targets ready for modeling.")


Feature shape: (100000, 10)
Targets ready for modeling.


In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout

# -------------------------------
# Assume df_model and processed features/targets exist
# X_scaled, y_campaign_score, y_performance_alerts, y_budget_rec, y_roi_forecast,
# y_budget_dist_total, y_avg_perf_score, y_budget_realloc, y_audience_exp, y_bid_strategy
# -------------------------------

# -------------------------------
# 1. Train-test split for features
# -------------------------------
X_train, X_test = train_test_split(X_scaled, test_size=0.2, random_state=42)

# -------------------------------
# 2. Train-test split for targets individually
# -------------------------------
y_campaign_score_train, y_campaign_score_test = train_test_split(y_campaign_score, test_size=0.2, random_state=42)
y_performance_alerts_train, y_performance_alerts_test = train_test_split(y_performance_alerts, test_size=0.2, random_state=42)
y_budget_rec_train, y_budget_rec_test = train_test_split(y_budget_rec, test_size=0.2, random_state=42)
y_roi_forecast_train, y_roi_forecast_test = train_test_split(y_roi_forecast, test_size=0.2, random_state=42)
y_budget_dist_train, y_budget_dist_test = train_test_split(y_budget_dist_total, test_size=0.2, random_state=42)
y_avg_perf_score_train, y_avg_perf_score_test = train_test_split(y_avg_perf_score, test_size=0.2, random_state=42)
y_budget_realloc_train, y_budget_realloc_test = train_test_split(y_budget_realloc, test_size=0.2, random_state=42)
y_audience_exp_train, y_audience_exp_test = train_test_split(y_audience_exp, test_size=0.2, random_state=42)
y_bid_strategy_train, y_bid_strategy_test = train_test_split(y_bid_strategy, test_size=0.2, random_state=42)

# -------------------------------
# 3. Combine targets into dicts
# -------------------------------
y_train_prepared = {
    'campaign_score': y_campaign_score_train,
    'roi_forecast': y_roi_forecast_train,
    'avg_perf_score': y_avg_perf_score_train,
    'budget_dist': y_budget_dist_train,
    'budget_realloc': y_budget_realloc_train,
    'performance_alerts': y_performance_alerts_train,
    'budget_rec': y_budget_rec_train,
    'audience_exp': y_audience_exp_train,
    'bid_strategy': y_bid_strategy_train
}

y_test_prepared = {
    'campaign_score': y_campaign_score_test,
    'roi_forecast': y_roi_forecast_test,
    'avg_perf_score': y_avg_perf_score_test,
    'budget_dist': y_budget_dist_test,
    'budget_realloc': y_budget_realloc_test,
    'performance_alerts': y_performance_alerts_test,
    'budget_rec': y_budget_rec_test,
    'audience_exp': y_audience_exp_test,
    'bid_strategy': y_bid_strategy_test
}

# -------------------------------
# 4. Build multi-task neural network
# -------------------------------
input_layer = Input(shape=(X_train.shape[1],), name='input_features')

# Shared hidden layers
x = Dense(256, activation='relu')(input_layer)
x = Dropout(0.2)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(64, activation='relu')(x)

# Output layers
campaign_score_out = Dense(1, name='campaign_score')(x)
roi_forecast_out = Dense(1, name='roi_forecast')(x)
avg_perf_score_out = Dense(1, name='avg_perf_score')(x)
budget_dist_out = Dense(3, name='budget_dist')(x)
budget_realloc_out = Dense(1, name='budget_realloc')(x)

performance_alerts_out = Dense(1, activation='sigmoid', name='performance_alerts')(x)
budget_rec_out = Dense(len(np.unique(y_budget_rec)), activation='softmax', name='budget_rec')(x)
audience_exp_out = Dense(1, activation='sigmoid', name='audience_exp')(x)
bid_strategy_out = Dense(len(np.unique(y_bid_strategy)), activation='softmax', name='bid_strategy')(x)

# Build model
model = Model(inputs=input_layer, outputs=[
    campaign_score_out, roi_forecast_out, avg_perf_score_out, budget_dist_out, budget_realloc_out,
    performance_alerts_out, budget_rec_out, audience_exp_out, bid_strategy_out
])

# -------------------------------
# 5. Compile model
# -------------------------------
model.compile(
    optimizer='adam',
    loss={
        'campaign_score': 'mse',
        'roi_forecast': 'mse',
        'avg_perf_score': 'mse',
        'budget_dist': 'mse',
        'budget_realloc': 'mse',
        'performance_alerts': 'binary_crossentropy',
        'budget_rec': 'sparse_categorical_crossentropy',
        'audience_exp': 'binary_crossentropy',
        'bid_strategy': 'sparse_categorical_crossentropy'
    },
    metrics={
        'campaign_score': 'mae',
        'roi_forecast': 'mae',
        'avg_perf_score': 'mae',
        'budget_dist': 'mae',
        'budget_realloc': 'mae',
        'performance_alerts': 'accuracy',
        'budget_rec': 'accuracy',
        'audience_exp': 'accuracy',
        'bid_strategy': 'accuracy'
    }
)

# -------------------------------
# 6. Train model
# -------------------------------
history = model.fit(
    X_train, y_train_prepared,
    validation_data=(X_test, y_test_prepared),
    epochs=50,
    batch_size=512
)

print("Model training completed successfully!")


Epoch 1/50
157/157 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - audience_exp_accuracy: 0.4969 - audience_exp_loss: 0.8865 - avg_perf_score_loss: 4635.2383 - avg_perf_score_mae: 47.6814 - bid_strategy_accuracy: 0.2518 - bid_strategy_loss: 1.9424 - budget_dist_loss: 516.4344 - budget_dist_mae: 18.7843 - budget_realloc_loss: 120.4466 - budget_realloc_mae: 7.4658 - budget_rec_accuracy: 0.4094 - budget_rec_loss: 1.2652 - campaign_score_loss: 4539.7031 - campaign_score_mae: 47.0475 - loss: 11383.0439 - performance_alerts_accuracy: 0.7683 - performance_alerts_loss: 0.4392 - roi_forecast_loss: 1547.6107 - roi_forecast_mae: 16.8306 - val_audience_exp_accuracy: 0.5020 - val_audience_exp_loss: 0.7013 - val_avg_perf_score_loss: 533.5765 - val_avg_perf_score_mae: 18.3595 - val_bid_strategy_accuracy: 0.2520 - val_bid_strategy_loss: 1.8839 - val_budget_dist_loss: 266.8152 - val_budget_dist_mae: 13.4957 - val_budget_realloc_loss: 25.6778 - val_budget_realloc_mae: 4.1473 - val_budget_rec_accuracy: 0.4557 - val_

In [9]:
# -------------------------------
# Multi-Task Neural Network Training Code
# -------------------------------
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# -------------------------------
# Assume df_model and processed features/targets exist:
# X_scaled (features), and targets:
# y_campaign_score, y_performance_alerts, y_budget_rec, y_roi_forecast,
# y_budget_dist_total (shape: n_samples x 3), y_avg_perf_score, 
# y_budget_realloc, y_audience_exp, y_bid_strategy
# -------------------------------

# 1. Train-test split for features
X_train, X_test = train_test_split(X_scaled, test_size=0.2, random_state=42)

# 2. Train-test split for each target
def split_target(y):
    return train_test_split(y, test_size=0.2, random_state=42)

y_campaign_score_train, y_campaign_score_test = split_target(y_campaign_score)
y_roi_forecast_train, y_roi_forecast_test = split_target(y_roi_forecast)
y_avg_perf_score_train, y_avg_perf_score_test = split_target(y_avg_perf_score)
y_budget_realloc_train, y_budget_realloc_test = split_target(y_budget_realloc)
y_budget_dist_train, y_budget_dist_test = split_target(y_budget_dist_total)  # shape: (n_samples,3)

y_performance_alerts_train, y_performance_alerts_test = split_target(y_performance_alerts)
y_budget_rec_train, y_budget_rec_test = split_target(y_budget_rec)
y_audience_exp_train, y_audience_exp_test = split_target(y_audience_exp)
y_bid_strategy_train, y_bid_strategy_test = split_target(y_bid_strategy)

# 3. Scale regression targets
scalers = {}

for name, y_train, y_test in [
    ('campaign_score', y_campaign_score_train, y_campaign_score_test),
    ('roi_forecast', y_roi_forecast_train, y_roi_forecast_test),
    ('avg_perf_score', y_avg_perf_score_train, y_avg_perf_score_test),
    ('budget_realloc', y_budget_realloc_train, y_budget_realloc_test)
]:
    scaler = StandardScaler()
    y_campaign_score_train_scaled = scaler.fit_transform(np.array(y_train).reshape(-1,1))
    y_campaign_score_test_scaled = scaler.transform(np.array(y_test).reshape(-1,1))
    if name == 'campaign_score':
        y_campaign_score_train, y_campaign_score_test = y_campaign_score_train_scaled, y_campaign_score_test_scaled
    elif name == 'roi_forecast':
        y_roi_forecast_train, y_roi_forecast_test = y_campaign_score_train_scaled, y_campaign_score_test_scaled
    elif name == 'avg_perf_score':
        y_avg_perf_score_train, y_avg_perf_score_test = y_campaign_score_train_scaled, y_campaign_score_test_scaled
    elif name == 'budget_realloc':
        y_budget_realloc_train, y_budget_realloc_test = y_campaign_score_train_scaled, y_campaign_score_test_scaled
    scalers[name] = scaler

# Scale multi-output regression: budget_dist (n_samples,3)
scaler_budget_dist = StandardScaler()
y_budget_dist_train = scaler_budget_dist.fit_transform(y_budget_dist_train)
y_budget_dist_test = scaler_budget_dist.transform(y_budget_dist_test)
scalers['budget_dist'] = scaler_budget_dist

# 4. Combine targets into dicts
y_train_prepared = {
    'campaign_score': y_campaign_score_train,
    'roi_forecast': y_roi_forecast_train,
    'avg_perf_score': y_avg_perf_score_train,
    'budget_dist': y_budget_dist_train,
    'budget_realloc': y_budget_realloc_train,
    'performance_alerts': y_performance_alerts_train,
    'budget_rec': y_budget_rec_train,
    'audience_exp': y_audience_exp_train,
    'bid_strategy': y_bid_strategy_train
}

y_test_prepared = {
    'campaign_score': y_campaign_score_test,
    'roi_forecast': y_roi_forecast_test,
    'avg_perf_score': y_avg_perf_score_test,
    'budget_dist': y_budget_dist_test,
    'budget_realloc': y_budget_realloc_test,
    'performance_alerts': y_performance_alerts_test,
    'budget_rec': y_budget_rec_test,
    'audience_exp': y_audience_exp_test,
    'bid_strategy': y_bid_strategy_test
}

# 5. Compute class weights for imbalanced classification targets
audience_exp_weights = compute_class_weight('balanced', classes=np.unique(y_audience_exp_train), y=y_audience_exp_train)
bid_strategy_weights = compute_class_weight('balanced', classes=np.unique(y_bid_strategy_train), y=y_bid_strategy_train)
class_weights = {
    'audience_exp': dict(enumerate(audience_exp_weights)),
    'bid_strategy': dict(enumerate(bid_strategy_weights))
}

# 6. Build multi-task neural network
input_layer = Input(shape=(X_train.shape[1],), name='input_features')

# Shared layers
x = Dense(256, activation='relu')(input_layer)
x = Dropout(0.2)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(64, activation='relu')(x)

# Regression outputs
campaign_score_out = Dense(32, activation='relu')(x)
campaign_score_out = Dense(1, name='campaign_score')(campaign_score_out)

roi_forecast_out = Dense(16, activation='relu')(x)
roi_forecast_out = Dense(1, name='roi_forecast')(roi_forecast_out)

avg_perf_score_out = Dense(16, activation='relu')(x)
avg_perf_score_out = Dense(1, name='avg_perf_score')(avg_perf_score_out)

budget_realloc_out = Dense(16, activation='relu')(x)
budget_realloc_out = Dense(1, name='budget_realloc')(budget_realloc_out)

budget_dist_out = Dense(16, activation='relu')(x)
budget_dist_out = Dense(3, name='budget_dist')(budget_dist_out)  # matches y shape (n_samples,3)

# Classification outputs
performance_alerts_out = Dense(16, activation='relu')(x)
performance_alerts_out = Dense(1, activation='sigmoid', name='performance_alerts')(performance_alerts_out)

budget_rec_out = Dense(32, activation='relu')(x)
budget_rec_out = Dense(len(np.unique(y_budget_rec_train)), activation='softmax', name='budget_rec')(budget_rec_out)

audience_exp_out = Dense(16, activation='relu')(x)
audience_exp_out = Dense(1, activation='sigmoid', name='audience_exp')(audience_exp_out)

bid_strategy_out = Dense(32, activation='relu')(x)
bid_strategy_out = Dense(len(np.unique(y_bid_strategy_train)), activation='softmax', name='bid_strategy')(bid_strategy_out)

# Build model
model = Model(inputs=input_layer, outputs=[
    campaign_score_out, roi_forecast_out, avg_perf_score_out, budget_dist_out, budget_realloc_out,
    performance_alerts_out, budget_rec_out, audience_exp_out, bid_strategy_out
])

# 7. Compile model
model.compile(
    optimizer='adam',
    loss={
        'campaign_score': 'mse',
        'roi_forecast': 'mse',
        'avg_perf_score': 'mse',
        'budget_dist': 'mse',
        'budget_realloc': 'mse',
        'performance_alerts': 'binary_crossentropy',
        'budget_rec': 'sparse_categorical_crossentropy',
        'audience_exp': 'binary_crossentropy',
        'bid_strategy': 'sparse_categorical_crossentropy'
    },
    metrics={
        'campaign_score': 'mae',
        'roi_forecast': 'mae',
        'avg_perf_score': 'mae',
        'budget_dist': 'mae',
        'budget_realloc': 'mae',
        'performance_alerts': 'accuracy',
        'budget_rec': 'accuracy',
        'audience_exp': 'accuracy',
        'bid_strategy': 'accuracy'
    },
    loss_weights={k:1.0 for k in y_train_prepared.keys()}
)

# 8. Callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-5)
]

# -------------------------------
# 9. Train model (without class_weight)
# -------------------------------
history = model.fit(
    X_train, y_train_prepared,
    validation_data=(X_test, y_test_prepared),
    epochs=50,
    batch_size=512,
    callbacks=callbacks,
    verbose=1
)


print("Multi-task model training completed successfully!")


Epoch 1/50
157/157 ━━━━━━━━━━━━━━━━━━━━ 9s 13ms/step - audience_exp_accuracy: 0.4972 - audience_exp_loss: 0.6973 - avg_perf_score_loss: 0.1380 - avg_perf_score_mae: 0.2689 - bid_strategy_accuracy: 0.2495 - bid_strategy_loss: 1.3935 - budget_dist_loss: 0.2286 - budget_dist_mae: 0.3293 - budget_realloc_loss: 1.0171 - budget_realloc_mae: 0.8722 - budget_rec_accuracy: 0.4915 - budget_rec_loss: 0.8083 - campaign_score_loss: 0.1403 - campaign_score_mae: 0.2683 - loss: 4.7888 - performance_alerts_accuracy: 0.9017 - performance_alerts_loss: 0.2571 - roi_forecast_loss: 0.1058 - roi_forecast_mae: 0.1696 - val_audience_exp_accuracy: 0.4949 - val_audience_exp_loss: 0.6946 - val_avg_perf_score_loss: 0.0299 - val_avg_perf_score_mae: 0.1384 - val_bid_strategy_accuracy: 0.2472 - val_bid_strategy_loss: 1.3881 - val_budget_dist_loss: 0.0114 - val_budget_dist_mae: 0.0796 - val_budget_realloc_loss: 0.9932 - val_budget_realloc_mae: 0.8645 - val_budget_rec_accuracy: 0.5214 - val_budget_rec_loss: 0.7292 - va

In [11]:
#try3
# -------------------------------
# Multi-Task Neural Network Training Code (Weighted for Imbalanced Classification)
# -------------------------------
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# -------------------------------
# Assume df_model and processed features/targets exist:
# X_scaled (features), and targets:
# y_campaign_score, y_performance_alerts, y_budget_rec, y_roi_forecast,
# y_budget_dist_total (shape: n_samples x 3), y_avg_perf_score, 
# y_budget_realloc, y_audience_exp, y_bid_strategy
# -------------------------------

# 1. Train-test split for features
X_train, X_test = train_test_split(X_scaled, test_size=0.2, random_state=42)

# 2. Train-test split for each target
def split_target(y):
    return train_test_split(y, test_size=0.2, random_state=42)

y_campaign_score_train, y_campaign_score_test = split_target(y_campaign_score)
y_roi_forecast_train, y_roi_forecast_test = split_target(y_roi_forecast)
y_avg_perf_score_train, y_avg_perf_score_test = split_target(y_avg_perf_score)
y_budget_realloc_train, y_budget_realloc_test = split_target(y_budget_realloc)
y_budget_dist_train, y_budget_dist_test = split_target(y_budget_dist_total)  # shape: (n_samples,3)

y_performance_alerts_train, y_performance_alerts_test = split_target(y_performance_alerts)
y_budget_rec_train, y_budget_rec_test = split_target(y_budget_rec)
y_audience_exp_train, y_audience_exp_test = split_target(y_audience_exp)
y_bid_strategy_train, y_bid_strategy_test = split_target(y_bid_strategy)

# 3. Scale regression targets
scalers = {}

for name, y_train, y_test in [
    ('campaign_score', y_campaign_score_train, y_campaign_score_test),
    ('roi_forecast', y_roi_forecast_train, y_roi_forecast_test),
    ('avg_perf_score', y_avg_perf_score_train, y_avg_perf_score_test),
    ('budget_realloc', y_budget_realloc_train, y_budget_realloc_test)
]:
    scaler = StandardScaler()
    y_train_scaled = scaler.fit_transform(np.array(y_train).reshape(-1,1))
    y_test_scaled = scaler.transform(np.array(y_test).reshape(-1,1))
    if name == 'campaign_score':
        y_campaign_score_train, y_campaign_score_test = y_train_scaled, y_test_scaled
    elif name == 'roi_forecast':
        y_roi_forecast_train, y_roi_forecast_test = y_train_scaled, y_test_scaled
    elif name == 'avg_perf_score':
        y_avg_perf_score_train, y_avg_perf_score_test = y_train_scaled, y_test_scaled
    elif name == 'budget_realloc':
        y_budget_realloc_train, y_budget_realloc_test = y_train_scaled, y_test_scaled
    scalers[name] = scaler

# Scale multi-output regression: budget_dist (n_samples,3)
scaler_budget_dist = StandardScaler()
y_budget_dist_train = scaler_budget_dist.fit_transform(y_budget_dist_train)
y_budget_dist_test = scaler_budget_dist.transform(y_budget_dist_test)
scalers['budget_dist'] = scaler_budget_dist

# 4. Combine targets into dicts
y_train_prepared = {
    'campaign_score': y_campaign_score_train,
    'roi_forecast': y_roi_forecast_train,
    'avg_perf_score': y_avg_perf_score_train,
    'budget_dist': y_budget_dist_train,
    'budget_realloc': y_budget_realloc_train,
    'performance_alerts': y_performance_alerts_train,
    'budget_rec': y_budget_rec_train,
    'audience_exp': y_audience_exp_train,
    'bid_strategy': y_bid_strategy_train
}

y_test_prepared = {
    'campaign_score': y_campaign_score_test,
    'roi_forecast': y_roi_forecast_test,
    'avg_perf_score': y_avg_perf_score_test,
    'budget_dist': y_budget_dist_test,
    'budget_realloc': y_budget_realloc_test,
    'performance_alerts': y_performance_alerts_test,
    'budget_rec': y_budget_rec_test,
    'audience_exp': y_audience_exp_test,
    'bid_strategy': y_bid_strategy_test
}

# 5. Compute per-sample weights for imbalanced classification
def get_sample_weights(y_train, classes):
    weights = compute_class_weight('balanced', classes=classes, y=y_train)
    return np.array([weights[int(label)] for label in y_train])

sample_weights = {
    'audience_exp': get_sample_weights(y_audience_exp_train, np.unique(y_audience_exp_train)),
    'bid_strategy': get_sample_weights(y_bid_strategy_train, np.unique(y_bid_strategy_train))
}

# 6. Build multi-task neural network
input_layer = Input(shape=(X_train.shape[1],), name='input_features')

# Shared layers
x = Dense(256, activation='relu')(input_layer)
x = Dropout(0.2)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(64, activation='relu')(x)

# Regression outputs
campaign_score_out = Dense(32, activation='relu')(x)
campaign_score_out = Dense(1, name='campaign_score')(campaign_score_out)

roi_forecast_out = Dense(16, activation='relu')(x)
roi_forecast_out = Dense(1, name='roi_forecast')(roi_forecast_out)

avg_perf_score_out = Dense(16, activation='relu')(x)
avg_perf_score_out = Dense(1, name='avg_perf_score')(avg_perf_score_out)

budget_realloc_out = Dense(16, activation='relu')(x)
budget_realloc_out = Dense(1, name='budget_realloc')(budget_realloc_out)

budget_dist_out = Dense(16, activation='relu')(x)
budget_dist_out = Dense(3, name='budget_dist')(budget_dist_out)  # matches y shape (n_samples,3)

# Classification outputs
performance_alerts_out = Dense(16, activation='relu')(x)
performance_alerts_out = Dense(1, activation='sigmoid', name='performance_alerts')(performance_alerts_out)

budget_rec_out = Dense(32, activation='relu')(x)
budget_rec_out = Dense(len(np.unique(y_budget_rec_train)), activation='softmax', name='budget_rec')(budget_rec_out)

audience_exp_out = Dense(16, activation='relu')(x)
audience_exp_out = Dense(1, activation='sigmoid', name='audience_exp')(audience_exp_out)

bid_strategy_out = Dense(32, activation='relu')(x)
bid_strategy_out = Dense(len(np.unique(y_bid_strategy_train)), activation='softmax', name='bid_strategy')(bid_strategy_out)

# Build model
model = Model(inputs=input_layer, outputs=[
    campaign_score_out, roi_forecast_out, avg_perf_score_out, budget_dist_out, budget_realloc_out,
    performance_alerts_out, budget_rec_out, audience_exp_out, bid_strategy_out
])

# 7. Compile model with adjusted loss weights
model.compile(
    optimizer='adam',
    loss={
        'campaign_score': 'mse',
        'roi_forecast': 'mse',
        'avg_perf_score': 'mse',
        'budget_dist': 'mse',
        'budget_realloc': 'mse',
        'performance_alerts': 'binary_crossentropy',
        'budget_rec': 'sparse_categorical_crossentropy',
        'audience_exp': 'binary_crossentropy',
        'bid_strategy': 'sparse_categorical_crossentropy'
    },
    metrics={
        'campaign_score': 'mae',
        'roi_forecast': 'mae',
        'avg_perf_score': 'mae',
        'budget_dist': 'mae',
        'budget_realloc': 'mae',
        'performance_alerts': 'accuracy',
        'budget_rec': 'accuracy',
        'audience_exp': 'accuracy',
        'bid_strategy': 'accuracy'
    },
    loss_weights={
        'campaign_score': 1.0,
        'roi_forecast': 1.0,
        'avg_perf_score': 1.0,
        'budget_dist': 1.0,
        'budget_realloc': 1.0,
        'performance_alerts': 1.0,
        'budget_rec': 1.0,
        'audience_exp': 3.0,      # increase weight for imbalanced binary task
        'bid_strategy': 3.0        # increase weight for imbalanced multi-class task
    }
)


# 8. Train model (no sample_weight)
history = model.fit(
    X_train, y_train_prepared,
    validation_data=(X_test, y_test_prepared),
    epochs=50,
    batch_size=512,
    callbacks=callbacks,
    verbose=1
)

# 8. Callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-5)
]

# -------------------------------
# 9. Train model without sample_weight
# -------------------------------
history = model.fit(
    X_train, y_train_prepared,
    validation_data=(X_test, y_test_prepared),
    epochs=50,
    batch_size=512,
    callbacks=callbacks,
    verbose=1
)

print("Multi-task model training completed successfully!")


print("Weighted multi-task model training completed successfully!")


Epoch 1/50
157/157 ━━━━━━━━━━━━━━━━━━━━ 9s 13ms/step - audience_exp_accuracy: 0.4991 - audience_exp_loss: 0.6972 - avg_perf_score_loss: 0.1380 - avg_perf_score_mae: 0.2653 - bid_strategy_accuracy: 0.2494 - bid_strategy_loss: 1.3931 - budget_dist_loss: 0.2085 - budget_dist_mae: 0.3170 - budget_realloc_loss: 1.0161 - budget_realloc_mae: 0.8715 - budget_rec_accuracy: 0.4743 - budget_rec_loss: 0.8301 - campaign_score_loss: 0.1466 - campaign_score_mae: 0.2678 - loss: 8.9611 - performance_alerts_accuracy: 0.9221 - performance_alerts_loss: 0.2365 - roi_forecast_loss: 0.1119 - roi_forecast_mae: 0.1604 - val_audience_exp_accuracy: 0.4915 - val_audience_exp_loss: 0.6940 - val_avg_perf_score_loss: 0.0285 - val_avg_perf_score_mae: 0.1346 - val_bid_strategy_accuracy: 0.2522 - val_bid_strategy_loss: 1.3886 - val_budget_dist_loss: 0.0091 - val_budget_dist_mae: 0.0720 - val_budget_realloc_loss: 0.9886 - val_budget_realloc_mae: 0.8620 - val_budget_rec_accuracy: 0.5155 - val_budget_rec_loss: 0.7257 - va

In [13]:
# -------------------------------
# Multi-Task Neural Network Training Code (Improved for Imbalanced Classification)
# -------------------------------
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.metrics import Precision, Recall

# -------------------------------
# 1. Train-test split for features
X_train, X_test = train_test_split(X_scaled, test_size=0.2, random_state=42)

# -------------------------------
# 2. Train-test split for each target
def split_target(y):
    return train_test_split(y, test_size=0.2, random_state=42)

y_campaign_score_train, y_campaign_score_test = split_target(y_campaign_score)
y_roi_forecast_train, y_roi_forecast_test = split_target(y_roi_forecast)
y_avg_perf_score_train, y_avg_perf_score_test = split_target(y_avg_perf_score)
y_budget_realloc_train, y_budget_realloc_test = split_target(y_budget_realloc)
y_budget_dist_train, y_budget_dist_test = split_target(y_budget_dist_total)

y_performance_alerts_train, y_performance_alerts_test = split_target(y_performance_alerts)
y_budget_rec_train, y_budget_rec_test = split_target(y_budget_rec)
y_audience_exp_train, y_audience_exp_test = split_target(y_audience_exp)
y_bid_strategy_train, y_bid_strategy_test = split_target(y_bid_strategy)

# -------------------------------
# 3. Scale regression targets
scalers = {}
for name, y_train, y_test in [
    ('campaign_score', y_campaign_score_train, y_campaign_score_test),
    ('roi_forecast', y_roi_forecast_train, y_roi_forecast_test),
    ('avg_perf_score', y_avg_perf_score_train, y_avg_perf_score_test),
    ('budget_realloc', y_budget_realloc_train, y_budget_realloc_test)
]:
    scaler = StandardScaler()
    y_train_scaled = scaler.fit_transform(np.array(y_train).reshape(-1,1))
    y_test_scaled = scaler.transform(np.array(y_test).reshape(-1,1))
    if name == 'campaign_score':
        y_campaign_score_train, y_campaign_score_test = y_train_scaled, y_test_scaled
    elif name == 'roi_forecast':
        y_roi_forecast_train, y_roi_forecast_test = y_train_scaled, y_test_scaled
    elif name == 'avg_perf_score':
        y_avg_perf_score_train, y_avg_perf_score_test = y_train_scaled, y_test_scaled
    elif name == 'budget_realloc':
        y_budget_realloc_train, y_budget_realloc_test = y_train_scaled, y_test_scaled
    scalers[name] = scaler

# Scale multi-output regression: budget_dist (n_samples,3)
scaler_budget_dist = StandardScaler()
y_budget_dist_train = scaler_budget_dist.fit_transform(y_budget_dist_train)
y_budget_dist_test = scaler_budget_dist.transform(y_budget_dist_test)
scalers['budget_dist'] = scaler_budget_dist

# -------------------------------
# 4. Compute sample weights for imbalanced classification
def get_sample_weights(y_train, classes):
    weights = compute_class_weight('balanced', classes=classes, y=y_train)
    return np.array([weights[int(label)] for label in y_train])

sample_weights = {
    'audience_exp': get_sample_weights(y_audience_exp_train, np.unique(y_audience_exp_train)),
    'bid_strategy': get_sample_weights(y_bid_strategy_train, np.unique(y_bid_strategy_train))
}

# -------------------------------
# 5. Build multi-task neural network
input_layer = Input(shape=(X_train.shape[1],), name='input_features')

# Shared layers
x = Dense(256, activation='relu')(input_layer)
x = Dropout(0.2)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(64, activation='relu')(x)

# Regression outputs
campaign_score_out = Dense(32, activation='relu')(x)
campaign_score_out = Dense(1, name='campaign_score')(campaign_score_out)

roi_forecast_out = Dense(16, activation='relu')(x)
roi_forecast_out = Dense(1, name='roi_forecast')(roi_forecast_out)

avg_perf_score_out = Dense(16, activation='relu')(x)
avg_perf_score_out = Dense(1, name='avg_perf_score')(avg_perf_score_out)

budget_realloc_out = Dense(16, activation='relu')(x)
budget_realloc_out = Dense(1, name='budget_realloc')(budget_realloc_out)

budget_dist_out = Dense(16, activation='relu')(x)
budget_dist_out = Dense(3, name='budget_dist')(budget_dist_out)

# Classification outputs
performance_alerts_out = Dense(16, activation='relu')(x)
performance_alerts_out = Dense(1, activation='sigmoid', name='performance_alerts')(performance_alerts_out)

budget_rec_out = Dense(32, activation='relu')(x)
budget_rec_out = Dense(len(np.unique(y_budget_rec_train)), activation='softmax', name='budget_rec')(budget_rec_out)

audience_exp_out = Dense(16, activation='relu')(x)
audience_exp_out = Dense(1, activation='sigmoid', name='audience_exp')(audience_exp_out)

bid_strategy_out = Dense(32, activation='relu')(x)
bid_strategy_out = Dense(len(np.unique(y_bid_strategy_train)), activation='softmax', name='bid_strategy')(bid_strategy_out)

# Build model
model = Model(inputs=input_layer, outputs=[
    campaign_score_out, roi_forecast_out, avg_perf_score_out, budget_dist_out, budget_realloc_out,
    performance_alerts_out, budget_rec_out, audience_exp_out, bid_strategy_out
])

# -------------------------------
# 6. Compile model with loss weights and per-class metrics
model.compile(
    optimizer='adam',
    loss={
        'campaign_score': 'mse',
        'roi_forecast': 'mse',
        'avg_perf_score': 'mse',
        'budget_dist': 'mse',
        'budget_realloc': 'mse',
        'performance_alerts': 'binary_crossentropy',
        'budget_rec': 'sparse_categorical_crossentropy',
        'audience_exp': 'binary_crossentropy',
        'bid_strategy': 'sparse_categorical_crossentropy'
    },
    metrics={
        'campaign_score': 'mae',
        'roi_forecast': 'mae',
        'avg_perf_score': 'mae',
        'budget_dist': 'mae',
        'budget_realloc': 'mae',
        'performance_alerts': ['accuracy', Precision(), Recall()],
        'budget_rec': ['accuracy'],
        'audience_exp': ['accuracy', Precision(), Recall()],
        'bid_strategy': ['accuracy']
    },
    loss_weights={
        'campaign_score': 1.0,
        'roi_forecast': 1.0,
        'avg_perf_score': 1.0,
        'budget_dist': 1.0,
        'budget_realloc': 1.0,
        'performance_alerts': 1.0,
        'budget_rec': 1.0,
        'audience_exp': 3.0,
        'bid_strategy': 3.0
    }
)

# -------------------------------
# 7. Callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-5)
]

# -------------------------------
# 8. Prepare y and sample_weights as lists for multi-output training
y_train_list = [
    y_campaign_score_train,
    y_roi_forecast_train,
    y_avg_perf_score_train,
    y_budget_dist_train,
    y_budget_realloc_train,
    y_performance_alerts_train,
    y_budget_rec_train,
    y_audience_exp_train,
    y_bid_strategy_train
]

y_test_list = [
    y_campaign_score_test,
    y_roi_forecast_test,
    y_avg_perf_score_test,
    y_budget_dist_test,
    y_budget_realloc_test,
    y_performance_alerts_test,
    y_budget_rec_test,
    y_audience_exp_test,
    y_bid_strategy_test
]

sample_weights_list = [
    None,  # campaign_score
    None,  # roi_forecast
    None,  # avg_perf_score
    None,  # budget_dist
    None,  # budget_realloc
    None,  # performance_alerts
    None,  # budget_rec
    sample_weights['audience_exp'],  # audience_exp
    sample_weights['bid_strategy']   # bid_strategy
]

# -------------------------------
# 9. Train model with per-output sample weights, increased epochs
history = model.fit(
    X_train, y_train_list,
    validation_data=(X_test, y_test_list),
    sample_weight=sample_weights_list,
    epochs=100,
    batch_size=512,
    callbacks=callbacks,
    verbose=1
)

print("Improved weighted multi-task model training completed successfully!")


AttributeError: 'NoneType' object has no attribute 'shape'

AttributeError: 'NoneType' object has no attribute 'shape'

In [9]:
# Multi-Task Neural Network Training Code (Stable Sample Weight Handling)
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.metrics import Precision, Recall

# -------------------------------
# Assume X_scaled and all targets are already defined
# X_scaled = ...
# y_campaign_score, y_roi_forecast, y_avg_perf_score, y_budget_realloc, y_budget_dist_total
# y_performance_alerts, y_budget_rec, y_audience_exp, y_bid_strategy

# -------------------------------
# 1. Train-test split for features
X_train, X_test = train_test_split(X_scaled, test_size=0.2, random_state=42)

# -------------------------------
# 2. Train-test split for each target
def split_target(y):
    return train_test_split(y, test_size=0.2, random_state=42)

(
    y_campaign_score_train, y_campaign_score_test,
    y_roi_forecast_train, y_roi_forecast_test,
    y_avg_perf_score_train, y_avg_perf_score_test,
    y_budget_realloc_train, y_budget_realloc_test,
    y_budget_dist_train, y_budget_dist_test,
    y_performance_alerts_train, y_performance_alerts_test,
    y_budget_rec_train, y_budget_rec_test,
    y_audience_exp_train, y_audience_exp_test,
    y_bid_strategy_train, y_bid_strategy_test
) = (
    *split_target(y_campaign_score),
    *split_target(y_roi_forecast),
    *split_target(y_avg_perf_score),
    *split_target(y_budget_realloc),
    *split_target(y_budget_dist_total),
    *split_target(y_performance_alerts),
    *split_target(y_budget_rec),
    *split_target(y_audience_exp),
    *split_target(y_bid_strategy)
)

# -------------------------------
# 2b. Flatten classification targets
for var in ['y_performance_alerts', 'y_budget_rec', 'y_audience_exp', 'y_bid_strategy']:
    locals()[f'{var}_train'] = np.ravel(locals()[f'{var}_train'])
    locals()[f'{var}_test'] = np.ravel(locals()[f'{var}_test'])

# -------------------------------
# 3. Scale regression targets
scalers = {}
for name, y_train, y_test in [
    ('campaign_score', y_campaign_score_train, y_campaign_score_test),
    ('roi_forecast', y_roi_forecast_train, y_roi_forecast_test),
    ('avg_perf_score', y_avg_perf_score_train, y_avg_perf_score_test),
    ('budget_realloc', y_budget_realloc_train, y_budget_realloc_test)
]:
    scaler = StandardScaler()
    y_train_scaled = scaler.fit_transform(np.array(y_train).reshape(-1,1))
    y_test_scaled = scaler.transform(np.array(y_test).reshape(-1,1))
    locals()[f'{name}_train'], locals()[f'{name}_test'] = y_train_scaled, y_test_scaled
    scalers[name] = scaler

# Multi-output regression: budget_dist (n_samples,3)
scaler_budget_dist = StandardScaler()
y_budget_dist_train = scaler_budget_dist.fit_transform(y_budget_dist_train)
y_budget_dist_test = scaler_budget_dist.transform(y_budget_dist_test)
scalers['budget_dist'] = scaler_budget_dist

# -------------------------------
# 4. Prepare targets list (order must match model outputs)
y_train_list = [
    y_campaign_score_train,
    y_roi_forecast_train,
    y_avg_perf_score_train,
    y_budget_dist_train,
    y_budget_realloc_train,
    y_performance_alerts_train,
    y_budget_rec_train,
    y_audience_exp_train,
    y_bid_strategy_train
]

y_test_list = [
    y_campaign_score_test,
    y_roi_forecast_test,
    y_avg_perf_score_test,
    y_budget_dist_test,
    y_budget_realloc_test,
    y_performance_alerts_test,
    y_budget_rec_test,
    y_audience_exp_test,
    y_bid_strategy_test
]

# -------------------------------
# 5. Class weights for classification outputs
audience_exp_class_weights = {0: 1.004, 1: 0.996}
bid_strategy_class_weights = {0: 1.001, 1: 0.996, 2: 0.998, 3: 1.005}

def apply_class_weights(y_train, class_weights):
    return np.array([class_weights[int(label)] for label in y_train])

# -------------------------------
# 6. Sample weights list aligned to outputs
sample_weights_list = [
    np.ones(len(y_campaign_score_train)),  # regression
    np.ones(len(y_roi_forecast_train)),    # regression
    np.ones(len(y_avg_perf_score_train)),  # regression
    np.ones(len(y_budget_dist_train)),     # regression
    np.ones(len(y_budget_realloc_train)),  # regression
    np.ones(len(y_performance_alerts_train)),  # binary
    np.ones(len(y_budget_rec_train)),      # multi-class
    apply_class_weights(y_audience_exp_train, audience_exp_class_weights),  # weighted binary
    apply_class_weights(y_bid_strategy_train, bid_strategy_class_weights)   # weighted multi-class
]

# -------------------------------
# 7. Build multi-task neural network
input_layer = Input(shape=(X_train.shape[1],), name='input_features')
x = Dense(256, activation='relu')(input_layer)
x = Dropout(0.2)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(64, activation='relu')(x)

# Regression outputs
campaign_score_out = Dense(32, activation='relu')(x)
campaign_score_out = Dense(1, name='campaign_score')(campaign_score_out)

roi_forecast_out = Dense(16, activation='relu')(x)
roi_forecast_out = Dense(1, name='roi_forecast')(roi_forecast_out)

avg_perf_score_out = Dense(16, activation='relu')(x)
avg_perf_score_out = Dense(1, name='avg_perf_score')(avg_perf_score_out)

budget_realloc_out = Dense(16, activation='relu')(x)
budget_realloc_out = Dense(1, name='budget_realloc')(budget_realloc_out)

budget_dist_out = Dense(16, activation='relu')(x)
budget_dist_out = Dense(3, name='budget_dist')(budget_dist_out)

# Classification outputs
performance_alerts_out = Dense(16, activation='relu')(x)
performance_alerts_out = Dense(1, activation='sigmoid', name='performance_alerts')(performance_alerts_out)

budget_rec_out = Dense(32, activation='relu')(x)
budget_rec_out = Dense(len(np.unique(y_budget_rec_train)), activation='softmax', name='budget_rec')(budget_rec_out)

audience_exp_out = Dense(16, activation='relu')(x)
audience_exp_out = Dense(1, activation='sigmoid', name='audience_exp')(audience_exp_out)

bid_strategy_out = Dense(32, activation='relu')(x)
bid_strategy_out = Dense(len(np.unique(y_bid_strategy_train)), activation='softmax', name='bid_strategy')(bid_strategy_out)

# Build model
model = Model(inputs=input_layer, outputs=[
    campaign_score_out, roi_forecast_out, avg_perf_score_out, budget_dist_out, budget_realloc_out,
    performance_alerts_out, budget_rec_out, audience_exp_out, bid_strategy_out
])

# -------------------------------
# 8. Compile model
model.compile(
    optimizer='adam',
    loss={
        'campaign_score': 'mse',
        'roi_forecast': 'mse',
        'avg_perf_score': 'mse',
        'budget_dist': 'mse',
        'budget_realloc': 'mse',
        'performance_alerts': 'binary_crossentropy',
        'budget_rec': 'sparse_categorical_crossentropy',
        'audience_exp': 'binary_crossentropy',
        'bid_strategy': 'sparse_categorical_crossentropy'
    },
    metrics={
        'campaign_score': 'mae',
        'roi_forecast': 'mae',
        'avg_perf_score': 'mae',
        'budget_dist': 'mae',
        'budget_realloc': 'mae',
        'performance_alerts': ['accuracy', Precision(), Recall()],
        'budget_rec': ['accuracy'],
        'audience_exp': ['accuracy', Precision(), Recall()],
        'bid_strategy': ['accuracy']
    },
    loss_weights={
        'campaign_score': 1.0,
        'roi_forecast': 1.0,
        'avg_perf_score': 1.0,
        'budget_dist': 1.0,
        'budget_realloc': 1.0,
        'performance_alerts': 1.0,
        'budget_rec': 1.0,
        'audience_exp': 3.0,
        'bid_strategy': 3.0
    }
)

# -------------------------------
# 9. Callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-5)
]

# -------------------------------
# 10. Train model with sample weights
history = model.fit(
    X_train, y_train_list,
    validation_data=(X_test, y_test_list),
    sample_weight=sample_weights_list,
    epochs=100,
    batch_size=512,
    callbacks=callbacks,
    verbose=1
)

print("Weighted multi-task model training completed successfully!")


Epoch 1/100
157/157 ━━━━━━━━━━━━━━━━━━━━ 10s 15ms/step - audience_exp_accuracy: 0.5021 - audience_exp_loss: 0.7614 - audience_exp_precision_1: 0.5047 - audience_exp_recall_1: 0.4749 - avg_perf_score_loss: 5346.2280 - avg_perf_score_mae: 52.2788 - bid_strategy_accuracy: 0.2494 - bid_strategy_loss: 1.5028 - budget_dist_loss: 1.1431 - budget_dist_mae: 0.8882 - budget_realloc_loss: 60.4737 - budget_realloc_mae: 5.9589 - budget_rec_accuracy: 0.4479 - budget_rec_loss: 1.0071 - campaign_score_loss: 4494.6177 - campaign_score_mae: 46.4024 - loss: 11478.5322 - performance_alerts_accuracy: 0.8988 - performance_alerts_loss: 0.3034 - performance_alerts_precision: 0.0919 - performance_alerts_recall: 0.0163 - roi_forecast_loss: 1525.1689 - roi_forecast_mae: 16.9670 - val_audience_exp_accuracy: 0.5001 - val_audience_exp_loss: 0.7086 - val_audience_exp_precision_1: 0.5005 - val_audience_exp_recall_1: 0.9636 - val_avg_perf_score_loss: 519.1801 - val_avg_perf_score_mae: 18.0214 - val_bid_strategy_accura

In [10]:
from sklearn.utils import class_weight
import numpy as np

targets = {
    "audience_exp": y_audience_exp,
    "bid_strategy": y_bid_strategy
}

class_weights_dict = {}

for name, y in targets.items():
    weights = class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(y), y=y)
    class_weights_dict[name] = dict(zip(np.unique(y), weights))
    print(f"{name} class weights: {class_weights_dict[name]}")


audience_exp class weights: {np.int64(0): np.float64(1.004157210852931), np.int64(1): np.float64(0.9958770689346107)}
bid_strategy class weights: {np.int64(0): np.float64(1.0008407061932023), np.int64(1): np.float64(0.9964129135113591), np.int64(2): np.float64(0.9980836793356755), np.int64(3): np.float64(1.0047020053852027)}


In [11]:
model.save("multi_task_model2.keras")


In [12]:
import numpy as np
from tensorflow import keras

# 1️⃣ Load the saved model
model = keras.models.load_model("multi_task_model.keras", compile=False)

# 2️⃣ Prepare a sample input
# Replace with the correct shape and dtype your model expects
sample_input = np.random.rand(1, input_feature_dim)  # e.g., (1, 20)

# 3️⃣ Get model predictions
predictions = model.predict(sample_input)

# 4️⃣ Print the outputs
# If it’s a multi-task model, predictions is usually a list of outputs
for i, output in enumerate(predictions):
    print(f"Output {i}: {output}")


NameError: name 'input_feature_dim' is not defined

In [13]:
# Load the model
from tensorflow import keras
model = keras.models.load_model("multi_task_model2.keras", compile=False)

# Check input shape
print(model.input_shape)


(None, 10)


In [14]:
import joblib

# Save StandardScaler
joblib.dump(scaler, "feature_scaler.save")

# Save LabelEncoders
joblib.dump(platform_encoder, "platform_encoder.save")
joblib.dump(budget_rec_encoder, "budget_rec_encoder.save")
joblib.dump(audience_exp_encoder, "audience_exp_encoder.save")
joblib.dump(bid_strategy_encoder, "bid_strategy_encoder.save")


['bid_strategy_encoder.save']

In [15]:
from tensorflow.keras.models import load_model

# Load Keras model
model = load_model("multi_task_model.keras")

# Load encoders and scaler
scaler = joblib.load("feature_scaler.save")
platform_encoder = joblib.load("platform_encoder.save")
budget_rec_encoder = joblib.load("budget_rec_encoder.save")
audience_exp_encoder = joblib.load("audience_exp_encoder.save")
bid_strategy_encoder = joblib.load("bid_strategy_encoder.save")


c:\Users\jyoti\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\saving\saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 44 variables whereas the saved optimizer has 86 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [41]:
import pandas as pd
import numpy as np
import re

def preprocess_row(row):
    # Encode platform
    row['platform_encoded'] = platform_encoder.transform([row['platform']])[0]

    # Process recommended_budget_distribution e.g., "22% Alpha, 38% Beta, 40% Gamma"
    parts = row['recommended_budget_distribution'].split(',')
    budget_alpha = float(parts[0].strip().split('%')[0])
    budget_beta  = float(parts[1].strip().split('%')[0])
    budget_gamma = float(parts[2].strip().split('%')[0])

    # Create input array
    X_input = np.array([[
        row['platform_encoded'], row['impressions'], row['clicks'], row['spend'],
        row['conversions'], row['CTR'], row['ROAS'],
        budget_alpha, budget_beta, budget_gamma
    ]])

    # Scale features
    X_scaled_input = scaler.transform(X_input)
    return X_scaled_input


In [42]:
def predict_row(row):
    X_scaled_input = preprocess_row(row)
    preds = model.predict(X_scaled_input)

    # preds is a list of 9 outputs, matching your model
    results = {
        'campaign_score': float(preds[0][0][0]),
        'roi_forecast': float(preds[1][0][0]),
        'avg_perf_score': float(preds[2][0][0]),
        'budget_dist': [float(x) for x in preds[3][0]],
        'budget_realloc': float(preds[4][0][0]),
        'performance_alerts': int(preds[5][0][0] > 0.5),
        'budget_rec': int(np.argmax(preds[6][0])),
        'audience_exp': int(preds[7][0][0] > 0.5),
        'bid_strategy': int(np.argmax(preds[8][0]))
    }

    # Optional: decode categorical outputs back to original labels
    results['budget_rec'] = budget_rec_encoder.inverse_transform([results['budget_rec']])[0]
    results['audience_exp'] = audience_exp_encoder.inverse_transform([results['audience_exp']])[0]
    results['bid_strategy'] = bid_strategy_encoder.inverse_transform([results['bid_strategy']])[0]

    return results


In [16]:
print(scaler.n_features_in_)


1


In [17]:
from sklearn.preprocessing import StandardScaler
import joblib

# X = features for training (10 columns)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)  # X has shape (n_samples, 10)

# Save the scaler for later use
joblib.dump(scaler, "feature_scaler.save")


['feature_scaler.save']

In [18]:
import joblib

scaler = joblib.load("feature_scaler.save")
print(scaler.n_features_in_)  # Should print 10


10


In [47]:
def preprocess_row(row):
    # Encode platform
    row['platform_encoded'] = platform_encoder.transform([row['platform']])[0]

    # Parse budget distribution
    budget_parts = row['recommended_budget_distribution'].split(',')
    budget_alpha = float(budget_parts[0].strip().split('%')[0])
    budget_beta  = float(budget_parts[1].strip().split('%')[0])
    budget_gamma = float(budget_parts[2].strip().split('%')[0])

    # Prepare feature array (10 features)
    X_input = np.array([[row['platform_encoded'], row['impressions'], row['clicks'], row['spend'],
                         row['conversions'], row['CTR'], row['ROAS'], budget_alpha, budget_beta, budget_gamma]])
    
    # Scale features using correct scaler
    X_scaled_input = scaler.transform(X_input)
    return X_scaled_input


In [48]:
X_scaled_input = preprocess_row(sample_row)
preds = model.predict(X_scaled_input)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 452ms/step


In [49]:
# Print raw predictions
print(preds)

# If preds is a list (multi-task model)
for i, p in enumerate(preds):
    print(f"Output {i+1}: {p}")


[array([[20.289515]], dtype=float32), array([[0.26728535]], dtype=float32), array([[19.938618]], dtype=float32), array([[-0.02508513,  0.0700812 , -0.00126377]], dtype=float32), array([[9.190035]], dtype=float32), array([[0.67218745]], dtype=float32), array([[0.69836885, 0.14677173, 0.15485948]], dtype=float32), array([[0.4760242]], dtype=float32), array([[0.24284509, 0.25327653, 0.26749656, 0.23638184]], dtype=float32)]
Output 1: [[20.289515]]
Output 2: [[0.26728535]]
Output 3: [[19.938618]]
Output 4: [[-0.02508513  0.0700812  -0.00126377]]
Output 5: [[9.190035]]
Output 6: [[0.67218745]]
Output 7: [[0.69836885 0.14677173 0.15485948]]
Output 8: [[0.4760242]]
Output 9: [[0.24284509 0.25327653 0.26749656 0.23638184]]


In [50]:
import numpy as np

def format_predictions(preds, budget_rec_encoder, bid_strategy_encoder, audience_exp_encoder):
    results = {}

    # 1. Campaign Score
    results["Campaign Score"] = float(preds[0][0][0])

    # 2. Performance Alerts (you may need mapping rules here)
    results["Performance Alerts"] = float(preds[1][0][0])  

    # 3. Budget Reallocation Opportunity
    results["Budget Reallocation (%)"] = float(preds[2][0][0])

    # 4. Bid Strategy Optimization (multi-class)
    bid_class = np.argmax(preds[3][0])
    results["Bid Strategy Optimization"] = bid_strategy_encoder.inverse_transform([bid_class])[0]

    # 5. ROI Forecast
    results["ROI Forecast (ROAS)"] = float(preds[4][0][0])

    # 6. Average Performance Score
    results["Avg Performance Score"] = float(preds[5][0][0])

    # 7. Budget Distribution (3-way softmax probs)
    dist = preds[6][0]
    results["Budget Distribution"] = {
        "Alpha": round(dist[0] * 100, 2),
        "Beta": round(dist[1] * 100, 2),
        "Gamma": round(dist[2] * 100, 2),
    }

    # 8. Audience Expansion Opportunity (binary prob)
    results["Audience Expansion"] = audience_exp_encoder.inverse_transform(
        [int(preds[7][0][0] > 0.5)]
    )[0]

    # 9. Budget Recommendation (multi-class)
    rec_class = np.argmax(preds[8][0])
    results["Budget Recommendation"] = budget_rec_encoder.inverse_transform([rec_class])[0]

    return results


In [51]:
pretty_preds = format_predictions(preds, budget_rec_encoder, bid_strategy_encoder, audience_exp_encoder)
for k, v in pretty_preds.items():
    print(f"{k}: {v}")


Campaign Score: 20.289514541625977
Performance Alerts: 0.2672853469848633
Budget Reallocation (%): 19.938617706298828
Bid Strategy Optimization: Maximize clicks
ROI Forecast (ROAS): 9.190034866333008
Avg Performance Score: 0.6721874475479126
Budget Distribution: {'Alpha': np.float32(69.84), 'Beta': np.float32(14.68), 'Gamma': np.float32(15.49)}
Audience Expansion: No
Budget Recommendation: Maintain current budget


In [52]:
def format_predictions(preds, budget_rec_encoder, bid_strategy_encoder, audience_exp_encoder):
    result = {}

    # Campaign Score (regression)
    result["Campaign Score"] = round(float(preds[0][0][0]), 2)

    # Performance Alerts (probability → label)
    result["Performance Alerts"] = "Triggered" if preds[1][0][0] > 0.5 else "Normal"

    # Budget Reallocation (%)
    result["Budget Reallocation (%)"] = round(float(preds[2][0][0]), 2)

    # Bid Strategy Optimization (classification)
    bid_idx = np.argmax(preds[3][0])
    result["Bid Strategy Optimization"] = bid_strategy_encoder.inverse_transform([bid_idx])[0]

    # ROI Forecast (ROAS)
    result["ROI Forecast (ROAS)"] = round(float(preds[4][0][0]), 2)

    # Avg Performance Score
    result["Avg Performance Score"] = round(float(preds[5][0][0]), 2)

    # Budget Distribution (softmax probabilities → %)
    dist = preds[6][0]
    result["Budget Distribution"] = {
        "Alpha": round(float(dist[0]) * 100, 2),
        "Beta": round(float(dist[1]) * 100, 2),
        "Gamma": round(float(dist[2]) * 100, 2),
    }

    # Audience Expansion (classification)
    aud_idx = np.argmax(preds[7][0])
    result["Audience Expansion"] = audience_exp_encoder.inverse_transform([aud_idx])[0]

    # Budget Recommendation (classification)
    rec_idx = np.argmax(preds[8][0])
    result["Budget Recommendation"] = budget_rec_encoder.inverse_transform([rec_idx])[0]

    return result


In [53]:
pretty_preds = format_predictions(preds, budget_rec_encoder, bid_strategy_encoder, audience_exp_encoder)
for k, v in pretty_preds.items():
    print(f"{k}: {v}")


Campaign Score: 20.29
Performance Alerts: Normal
Budget Reallocation (%): 19.94
Bid Strategy Optimization: Maximize clicks
ROI Forecast (ROAS): 9.19
Avg Performance Score: 0.67
Budget Distribution: {'Alpha': 69.84, 'Beta': 14.68, 'Gamma': 15.49}
Audience Expansion: No
Budget Recommendation: Maintain current budget


In [61]:
def format_predictions(preds, budget_rec_encoder, bid_strategy_encoder, audience_exp_encoder):
    result = {}

    def safe_val(x):
        import numpy as np
        if isinstance(x, (float, int, np.floating, np.integer)):
            return float(x)
        if hasattr(x, "ndim"):
            return float(x.ravel()[0])
        return x  # <-- return as-is if string

    result["Campaign Score"] = round(safe_val(preds['campaign_score']), 2)
    result["Performance Alerts"] = "Triggered" if safe_val(preds['performance_alerts']) > 0.5 else "Normal"
    result["Budget Reallocation (%)"] = round(safe_val(preds['budget_realloc']), 2)

    dist = preds['budget_dist']
    if hasattr(dist, "ndim"):
        dist = dist.ravel() * 100
        result["Budget Distribution"] = {
            "Alpha": round(float(dist[0]), 2),
            "Beta": round(float(dist[1]), 2),
            "Gamma": round(float(dist[2]), 2)
        }

    result["ROI Forecast (ROAS)"] = round(safe_val(preds['roi_forecast']), 2)
    result["Avg Performance Score"] = round(safe_val(preds['avg_perf_score']), 2)

    budget_rec_idx = np.argmax(preds['budget_rec'])
    result["Budget Recommendation"] = budget_rec_encoder.inverse_transform([budget_rec_idx])[0]

    # 🔹 Here we handle both prob or already-decoded string
    if isinstance(preds['audience_exp'], str):
        result["Audience Expansion"] = preds['audience_exp']
    else:
        result["Audience Expansion"] = "Yes" if safe_val(preds['audience_exp']) > 0.5 else "No"

    bid_idx = np.argmax(preds['bid_strategy'])
    result["Bid Strategy Optimization"] = bid_strategy_encoder.inverse_transform([bid_idx])[0]

    return result


In [97]:
import numpy as np

def format_predictions(preds, budget_rec_encoder, bid_strategy_encoder, audience_exp_encoder):
    result = {}

    # Helper to safely convert numeric values
    def safe_val(x):
        if isinstance(x, (float, int, np.floating, np.integer)):
            return float(x)
        if hasattr(x, "ndim"):  # numpy array / tf.Tensor
            return float(x.ravel()[0])
        return x  # if already string, return as-is

    # Regression outputs
    result["Campaign Score"] = round(safe_val(preds['campaign_score']), 2)
    result["Performance Alerts"] = "Triggered" if safe_val(preds['performance_alerts']) > 0.5 else "Normal"
    result["Budget Reallocation (%)"] = round(safe_val(preds['budget_realloc']), 2)
    result["ROI Forecast (ROAS)"] = round(safe_val(preds['roi_forecast']), 2)
    result["Avg Performance Score"] = round(safe_val(preds['avg_perf_score']), 2)

    # Budget Distribution
    dist = preds['budget_dist']
    if hasattr(dist, "ndim"):
        dist = dist.ravel() * 100
    result["Budget Distribution"] = {
        "Alpha": round(float(dist[0]), 2),
        "Beta": round(float(dist[1]), 2),
        "Gamma": round(float(dist[2]), 2)
    }

    # Budget Recommendation (multi-class)
    budget_rec_idx = np.argmax(preds['budget_rec'])
    result["Budget Recommendation"] = budget_rec_encoder.inverse_transform([budget_rec_idx])[0]

    # Audience Expansion (binary, handles string or prob)
    if isinstance(preds['audience_exp'], str):
        result["Audience Expansion"] = preds['audience_exp']
    else:
        result["Audience Expansion"] = "Yes" if safe_val(preds['audience_exp']) > 0.5 else "No"

    # Bid Strategy Optimization (multi-class)
    bid_idx = np.argmax(preds['bid_strategy'])
    result["Bid Strategy Optimization"] = bid_strategy_encoder.inverse_transform([bid_idx])[0]

    return result


In [113]:
sample_row = df.iloc[211].copy()        # pick any row
preds = predict_row(sample_row)        # returns raw or partially decoded outputs
pretty_preds = format_predictions(preds, budget_rec_encoder, bid_strategy_encoder, audience_exp_encoder)

for k, v in pretty_preds.items():
    print(f"{k}: {v}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step
Campaign Score: 193.77
Performance Alerts: Normal
Budget Reallocation (%): 10.23
ROI Forecast (ROAS): 7.56
Avg Performance Score: 196.9
Budget Distribution: {'Alpha': 1.4, 'Beta': 6.9, 'Gamma': 1.45}
Budget Recommendation: Maintain current budget
Audience Expansion: No
Bid Strategy Optimization: Maximize clicks


In [114]:
def predict_row(row):
    """
    Predict all 9 outputs for a single CSV row.
    Returns a dict keyed by output names, compatible with format_predictions().
    """
    # --- Encode platform ---
    row = row.copy()  # avoid SettingWithCopyWarning
    if 'platform_encoded' not in row:
        row['platform_encoded'] = platform_encoder.transform([row['platform']])[0]

    # --- Process recommended_budget_distribution ---
    if isinstance(row['recommended_budget_distribution'], str):
        parts = row['recommended_budget_distribution'].split(',')
        percentages = []
        for part in parts:
            try:
                percent = float(part.strip().split('%')[0])
            except:
                percent = 0.0
            percentages.append(percent)
        budget_alpha, budget_beta, budget_gamma = percentages
    else:
        budget_alpha = budget_beta = budget_gamma = 0.0

    # --- Prepare feature array ---
    X_input = np.array([[row['platform_encoded'], row['impressions'], row['clicks'], row['spend'],
                         row['conversions'], row['CTR'], row['ROAS'],
                         budget_alpha, budget_beta, budget_gamma]])
    
    # --- Scale features (using the scaler from training) ---
    X_scaled_input = scaler.transform(X_input)

    # --- Predict ---
    preds_list = model.predict(X_scaled_input)

    # --- Convert to dict keyed by output names ---
    output_names = ['campaign_score', 'roi_forecast', 'avg_perf_score', 'budget_dist',
                    'budget_realloc', 'performance_alerts', 'budget_rec',
                    'audience_exp', 'bid_strategy']

    preds_dict = {name: pred for name, pred in zip(output_names, preds_list)}
    return preds_dict


In [119]:
sample_row = df.iloc[8790].copy()            # pick any row
preds = predict_row(sample_row)            # returns dict keyed by output names
pretty_preds = format_predictions(preds, budget_rec_encoder, bid_strategy_encoder, audience_exp_encoder)

for k, v in pretty_preds.items():
    print(f"{k}: {v}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
Campaign Score: 24.69
Performance Alerts: Normal
Budget Reallocation (%): 10.05
ROI Forecast (ROAS): 0.12
Avg Performance Score: 23.89
Budget Distribution: {'Alpha': -1.51, 'Beta': 9.83, 'Gamma': -2.95}
Budget Recommendation: Decrease budget or pause underperforming ads
Audience Expansion: No
Bid Strategy Optimization: Maximize conversions


In [120]:
def predict_and_display(row):
    """
    Input: row (pandas Series or dict) representing a single campaign
    Output: prints all 9 predictions in a human-readable format
    """
    # --- Ensure row is a copy to avoid warnings ---
    row = row.copy()

    # --- Encode platform ---
    if 'platform_encoded' not in row:
        row['platform_encoded'] = platform_encoder.transform([row['platform']])[0]

    # --- Process recommended_budget_distribution ---
    if isinstance(row['recommended_budget_distribution'], str):
        parts = row['recommended_budget_distribution'].split(',')
        percentages = []
        for part in parts:
            try:
                percent = float(part.strip().split('%')[0])
            except:
                percent = 0.0
            percentages.append(percent)
        budget_alpha, budget_beta, budget_gamma = percentages
    else:
        budget_alpha = budget_beta = budget_gamma = 0.0

    # --- Prepare features and scale ---
    X_input = np.array([[row['platform_encoded'], row['impressions'], row['clicks'], row['spend'],
                         row['conversions'], row['CTR'], row['ROAS'],
                         budget_alpha, budget_beta, budget_gamma]])
    X_scaled_input = scaler.transform(X_input)

    # --- Predict using model ---
    preds_list = model.predict(X_scaled_input)

    # --- Map predictions to output names ---
    output_names = ['campaign_score', 'roi_forecast', 'avg_perf_score', 'budget_dist',
                    'budget_realloc', 'performance_alerts', 'budget_rec',
                    'audience_exp', 'bid_strategy']
    preds_dict = {name: pred for name, pred in zip(output_names, preds_list)}

    # --- Format predictions for readability ---
    def safe_val(x):
        """Convert array/tensor to float safely."""
        if hasattr(x, "ndim"):
            return float(x.ravel()[0])
        try:
            return float(x)
        except:
            return x  # already decoded string

    result = {}

    # Campaign Score
    result["Campaign Score"] = round(safe_val(preds_dict['campaign_score']), 2)

    # Performance Alerts
    result["Performance Alerts"] = "Triggered" if safe_val(preds_dict['performance_alerts']) > 0.5 else "Normal"

    # Budget Reallocation
    result["Budget Reallocation (%)"] = round(safe_val(preds_dict['budget_realloc']), 2)

    # Bid Strategy Optimization
    bid_idx = np.argmax(preds_dict['bid_strategy'])
    result["Bid Strategy Optimization"] = bid_strategy_encoder.inverse_transform([bid_idx])[0]

    # ROI Forecast
    result["ROI Forecast (ROAS)"] = round(safe_val(preds_dict['roi_forecast']), 2)

    # Avg Performance Score
    result["Avg Performance Score"] = round(safe_val(preds_dict['avg_perf_score']), 2)

    # Budget Distribution
    bd = preds_dict['budget_dist'].ravel()
    result["Budget Distribution"] = {
        'Alpha': round(float(bd[0]), 2),
        'Beta': round(float(bd[1]), 2),
        'Gamma': round(float(bd[2]), 2)
    }

    # Audience Expansion
    result["Audience Expansion"] = "Yes" if safe_val(preds_dict['audience_exp']) > 0.5 else "No"

    # Budget Recommendation
    budget_idx = np.argmax(preds_dict['budget_rec'])
    result["Budget Recommendation"] = budget_rec_encoder.inverse_transform([budget_idx])[0]

    # --- Print nicely ---
    print("=== Campaign Prediction ===")
    for k, v in result.items():
        print(f"{k}: {v}")

    return result


In [ ]:

sample_row = df.iloc[87]


predictions = predict_and_display(sample_row)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
=== Campaign Prediction ===
Campaign Score: 222.32
Performance Alerts: Normal
Budget Reallocation (%): 10.63
Bid Strategy Optimization: Maximize clicks
ROI Forecast (ROAS): 36.55
Avg Performance Score: 222.76
Budget Distribution: {'Alpha': -0.16, 'Beta': 0.02, 'Gamma': 0.07}
Audience Expansion: No
Budget Recommendation: Maintain current budget


In [19]:
def predict_and_display(row):
    """
    Input: row (pandas Series or dict) representing a single campaign
    Output: prints all 9 predictions in a human-readable format
    """
    # --- Ensure row is a copy to avoid warnings ---
    row = row.copy()

    # --- Encode platform ---
    if 'platform_encoded' not in row:
        row['platform_encoded'] = platform_encoder.transform([row['platform']])[0]

    # --- Process recommended_budget_distribution ---
    if isinstance(row['recommended_budget_distribution'], str):
        parts = row['recommended_budget_distribution'].split(',')
        percentages = []
        for part in parts:
            try:
                percent = float(part.strip().split('%')[0])
            except:
                percent = 0.0
            percentages.append(percent)
        budget_alpha, budget_beta, budget_gamma = percentages
    else:
        budget_alpha = budget_beta = budget_gamma = 0.0

    # --- Prepare features and scale ---
    X_input = np.array([[row['platform_encoded'], row['impressions'], row['clicks'], row['spend'],
                         row['conversions'], row['CTR'], row['ROAS'],
                         budget_alpha, budget_beta, budget_gamma]])
    X_scaled_input = scaler.transform(X_input)

    # --- Predict using model ---
    preds_list = model.predict(X_scaled_input)

    # --- Map predictions to output names ---
    output_names = ['campaign_score', 'roi_forecast', 'avg_perf_score', 'budget_dist',
                    'budget_realloc', 'performance_alerts', 'budget_rec',
                    'audience_exp', 'bid_strategy']
    preds_dict = {name: pred for name, pred in zip(output_names, preds_list)}

    # --- Helper for safely converting outputs ---
    def safe_val(x):
        """Convert array/tensor to float if possible, else return as-is"""
        if hasattr(x, "ndim"):
            return float(x.ravel()[0])
        try:
            return float(x)
        except:
            return x  # already decoded

    # --- Prepare results ---
    result = {}

    result["Campaign Score"] = round(safe_val(preds_dict['campaign_score']), 2)
    result["Performance Alerts"] = "Triggered" if safe_val(preds_dict['performance_alerts']) > 0.5 else "Normal"
    result["Budget Reallocation (%)"] = round(safe_val(preds_dict['budget_realloc']), 2)

    bid_idx = np.argmax(preds_dict['bid_strategy'])
    result["Bid Strategy Optimization"] = bid_strategy_encoder.inverse_transform([bid_idx])[0]

    result["ROI Forecast (ROAS)"] = round(safe_val(preds_dict['roi_forecast']), 2)
    result["Avg Performance Score"] = round(safe_val(preds_dict['avg_perf_score']), 2)

    # Budget Distribution (inverse scaled)
    bd_scaled = preds_dict['budget_dist']
    bd_actual = scaler_budget_dist.inverse_transform(bd_scaled)
    result["Budget Distribution"] = {
        'Alpha': round(float(bd_actual[0,0]), 2),
        'Beta': round(float(bd_actual[0,1]), 2),
        'Gamma': round(float(bd_actual[0,2]), 2)
    }

    result["Audience Expansion"] = "Yes" if safe_val(preds_dict['audience_exp']) > 0.5 else "No"

    budget_idx = np.argmax(preds_dict['budget_rec'])
    result["Budget Recommendation"] = budget_rec_encoder.inverse_transform([budget_idx])[0]

    # --- Print nicely ---
    print("=== Campaign Prediction ===")
    for k, v in result.items():
        print(f"{k}: {v}")


In [25]:
predict_and_display(df.iloc[221])


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
=== Campaign Prediction ===
Campaign Score: 37.39
Performance Alerts: Normal
Budget Reallocation (%): 9.1
Bid Strategy Optimization: Maximize clicks
ROI Forecast (ROAS): 0.71
Avg Performance Score: 38.04
Budget Distribution: {'Alpha': 34.62, 'Beta': 34.3, 'Gamma': 31.4}
Audience Expansion: No
Budget Recommendation: Increase budget


In [28]:
for i in range(5): 
    print(f"\n--- Row {i} ---")
    predict_and_display(df.iloc[i])



--- Row 0 ---
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 233ms/step
=== Campaign Prediction ===
Campaign Score: 20.29
Performance Alerts: Triggered
Budget Reallocation (%): 9.19
Bid Strategy Optimization: Maximize conversions
ROI Forecast (ROAS): 0.27
Avg Performance Score: 19.94
Budget Distribution: {'Alpha': 34.81, 'Beta': 35.59, 'Gamma': 29.98}
Audience Expansion: No
Budget Recommendation: Decrease budget or pause underperforming ads

--- Row 1 ---
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
=== Campaign Prediction ===
Campaign Score: 172.89
Performance Alerts: Normal
Budget Reallocation (%): 9.51
Bid Strategy Optimization: Maximize clicks
ROI Forecast (ROAS): 9.83
Avg Performance Score: 174.52
Budget Distribution: {'Alpha': 33.89, 'Beta': 34.42, 'Gamma': 31.58}
Audience Expansion: No
Budget Recommendation: Maintain current budget

--- Row 2 ---
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
=== Campaign Prediction ===
Campaign Score: 156.07
Performance Alerts: Normal
Budget Reallocation (%): 10.56
Bid Strategy

In [1]:
import joblib

platform_encoder = joblib.load(r"C:\Users\jyoti\OneDrive\Desktop\ASF Hackathon\lasttry\platform_encoder.save")
print(platform_encoder.classes_)


['Facebook Ads' 'Google Ads' 'Instagram Ads' 'LinkedIn Ads']
